In [1]:
def get_top100_list():
    output_list = list()
    url = "https://www.billboard.com/charts/hot-100"

    import requests
    from bs4 import BeautifulSoup
    
    try:
        response = requests.get(url)   
        if not response.status_code == 200:
            print("HTTP error",response.status_code)
        else:
            try:
                soup = BeautifulSoup(response.content,'lxml')
            except:
                print('something went wrong')
    except:
        print("Something went wrong with request.get")
    
    top1_name = soup.find('div',class_='chart-number-one__title').get_text()
    top1_artist = soup.find('div',class_='chart-number-one__artist').get_text()
    output_list.append((top1_name,top1_artist))
    
    all_songs = soup.find_all('div',class_='chart-list-item__text')
    
    for song in all_songs:
        song_name = song.find('span',class_='chart-list-item__title-text').get_text()
        artist = song.find('div',class_='chart-list-item__artist').get_text()
        output_list.append((song_name,artist))
    
    return output_list

In [2]:
def get_all_info(list_of_songs):
    import requests
    import re
    from bs4 import BeautifulSoup
    all_song_info = []
    for song in list_of_songs:
        name = song[0].strip()
        string = 'Sunflower (Spider-Man: Into The Spider-Verse)'
        pattern = r'[^()]+'
        match = re.search(pattern,name)
        name = match.group().strip()
        artist = song[1].strip()
        token = 'agZ_VYrkzow8Wo80yQSUpgi0V9J9szWwtLF4cY9inzE-jIoOe3xrs43F9yYB28Xg'
        base_url = 'https://api.genius.com'
        headers = {'Authorization': 'Bearer ' + token}
        search_url = base_url + '/search'
        data = {'q': name + ' ' + artist}
        try:
            response = requests.get(search_url, data=data, headers=headers)
        except:
            print('Error happens.')
        lyric_path = 'http://genius.com'+response.json()['response']['hits'][0]['result']['path']
        try:
            response2 = requests.get(lyric_path)
        except:
            print('Error happens while searching for lyrics.')
            continue
        response2_page = BeautifulSoup(response2.content,'lxml')
        list_=response2_page.find('div',class_='lyrics').find_all('a',class_='referent')
        lyrics = ''
        for item in list_:
            line = item.get_text()
            lyrics += line + ' '
        all_song_info.append((name,artist,lyrics))
    return all_song_info

### search songs

In [5]:
def get_song_lyrics(top100_list):  # top100_list contain (song_name,artist,lyrics)
    i=0
    song_lyrics_dict = dict()
    while i < len(top100_list):
        song_name = top100_list[i][0].strip()
        lyrics = top100_list[i][2].strip()
        song_lyrics_dict[song_name]= lyrics
        i = i+1
    return song_lyrics_dict # (song_name,lyrics)

In [ ]:
song_lyrics_dict = get_song_lyrics(top100_list)

In [ ]:
def find_song(search_list):  # song_lyrics_dict
    
    import re
    from nltk import word_tokenize
    
    song_name= []
    result_dict=dict()
    for key in song_lyrics_dict.keys():
        result_dict[key] = 1
    
    for sword in search_list:
        sword = sword.lower()
        for key,value in song_lyrics_dict.items():
            lyrics=value.replace(',','').lower()
            lyrics_list = word_tokenize(lyrics)
            if sword not in lyrics_list:
                result_dict[key]= 0
    
    for key, value in result_dict.items():
        if value:
            song_name.append(key)
    return song_name

In [ ]:
find_song(['wreck'])